# LangChain: Models, Prompts and Output Parsers

## Disclaimer

This example is modified version of a jupyter notebook associate to the course [https://learn.deeplearning.ai/langchain/lesson/1/introduction](https://learn.deeplearning.ai/langchain/lesson/1/introduction).

In this version I adapted to run with Azure OpenaiKey, so I've made really small modidfication, I suggest you to do the original course.



## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

You can find information on how to create an OpenAI deployment on Azure in the official OpenAI API documentation. Here are the general steps:

1. Create an Azure account and sign in to the Azure portal.

2. Create a new resource group for your deployment.

3. Create a new Azure Kubernetes Service (AKS) cluster in your resource group.

4. Install the OpenAI Kubernetes operator on your AKS cluster.

5. Create a new OpenAI API deployment using the operator.

6. Configure your deployment by setting environment variables and other parameters.

7. Test your deployment by sending requests to the API endpoint.

For more detailed information on each of these steps, please refer to the official OpenAI API documentation on deploying to Azure: https://beta.openai.com/docs/deployments/azure

In [ ]:
#!pip install python-dotenv
#!pip install openai

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"

# Remember that you need to set the OPENAI_API_BASE to point openai to your specific deployment.

openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")

# print base to verify you are pointint to the right deployment
print(openai.api_base)


https://alkopenai.openai.azure.com/


## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
        engine="Gpt35" 
    )
    return response.choices[0].message["content"]


Now I can do a simple call to verify that we can call correctly deployment

In [3]:
get_completion("What is 1+1?")

'As an AI language model, I can tell you that the answer to 1+1 is 2.'

In [4]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [5]:
style = """American English \
in a calm and respectful tone
"""

In [6]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [7]:
from pprint import pprint
response = get_completion(prompt)
pprint(response, width=80)

('I am quite upset that my blender lid came off and caused my smoothie to '
 'splatter all over my kitchen walls. Additionally, the warranty does not '
 'cover the cost of cleaning up the mess. Would you be able to assist me at '
 'this time, my friend? Thank you kindly.')


## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [ ]:
#!pip install --upgrade langchain

### Model

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI

In [9]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0

# Following code is how you can call the standard openai api
# chat = ChatOpenAI(temperature=0.0)

# We need to create azure chat api key and endpoint SUPER IMPORTANT
# YOU NEED TO USE THE NAME OF YOUR CURRENT DEPLOYMENT AND THIS IS MUST
# BE A DEPLOYMENT OF A MODEL THAT IS COMPATIBLE WITH THE ENGINE 
# SPECIFIED IN UPPER CELL
chat = AzureChatOpenAI(deployment_name="gpt35", openai_api_version="2023-03-15-preview")
chat

AzureChatOpenAI(verbose=False, callbacks=None, callback_manager=None, tags=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.7, model_kwargs={}, openai_api_key='f652566301f243ae89af1f4cd7cb2707', openai_api_base='https://alkopenai.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, deployment_name='gpt35', openai_api_type='azure', openai_api_version='2023-03-15-preview')

### Prompt template

In [10]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [11]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [12]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True)

In [13]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [14]:
customer_style = """American English \
in a calm and respectful tone
"""

In [15]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [16]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [17]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain.schema.HumanMessage'>


In [18]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} example=False


Now we can simply run everything with the chat, remember we are using AZURE openai endpoint.

In [19]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

Actually we can print the message

In [20]:
pprint(customer_response.content)

("I am frustrated that my blender's lid came off and caused my smoothie to "
 "splatter all over my kitchen walls. Additionally, I'm disappointed to find "
 "out that the warranty won't cover the cost of cleaning up the mess. Could "
 'you please assist me with this situation? Thank you kindly.')


In [34]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [35]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [36]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

pprint(service_messages[0].content)

('Translate the text that is delimited by triple backticks into a style that '
 'is a polite tone that speaks in English Pirate. text: ```Hey there customer, '
 "the warranty does not cover cleaning expenses for your kitchen because it's "
 'your fault that you misused your blender by forgetting to put the lid on '
 'before starting the blender. Tough luck! See ya!\n'
 '```\n')


In [38]:
service_response = chat(service_messages)
pprint(service_response.content)

("Ahoy matey! Beggin' yer pardon, but the warranty doesn't be coverin' the "
 "cost of cleanin' yer galley due to ye own misuse of yer blender. Ye forgot "
 "to put the lid on before ye started blendin'! Arr, tough break me hearty! "
 'Fair winds to ye!')


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [39]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [40]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [43]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
pprint(vars(prompt_template))

{'input_variables': ['text'],
 'messages': [HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})],
 'output_parser': None,
 'partial_variables': {}}


In [54]:
messages = prompt_template.format_messages(text=customer_review)
pprint(messages)


[HumanMessage(content="For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.\n\n", additiona

In [55]:
# chat = ChatOpenAI(temperature=0.0)
chat = AzureChatOpenAI(deployment_name="gpt35", openai_api_version="2023-03-15-preview")
response = chat(messages)
pprint(response.content)


('{\n'
 '    "gift": true,\n'
 '    "delivery_days": 2,\n'
 '    "price_value": ["It\'s slightly more expensive than the other leaf '
 'blowers out there, but I think it\'s worth it for the extra features."]\n'
 '}')


You can **manual parse** but we will see after, that langchain can do this for us

To parse and pretty print the JSON content of the response, you can use the `json` module in Python. Here's an example:



In [57]:
import json

# Assuming response.content is a JSON string
json_content = json.loads(response.content)

# Use pprint to pretty print the JSON content with word wrapping
pprint(json_content, width=80)

{'delivery_days': 2,
 'gift': True,
 'price_value': ["It's slightly more expensive than the other leaf blowers out "
                 "there, but I think it's worth it for the extra features."]}


In [58]:
type(response.content)

str

In [59]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [60]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [61]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")


The code above defines three `ResponseSchema` objects that are used to extract specific information from text data. Each `ResponseSchema` object has a `name` and a `description` attribute that provide information about the type of information that should be extracted.

The first `ResponseSchema` object is named `gift_schema` and is used to extract information about whether an item was purchased as a gift for someone else. The `description` attribute provides instructions to answer `True` if the item was purchased as a gift, and `False` if it was not or if the information is unknown.

The second `ResponseSchema` object is named `delivery_days_schema` and is used to extract information about how many days it took for a product to arrive. The `description` attribute provides instructions to output `-1` if this information is not found.

The third `ResponseSchema` object is named `price_value_schema` and is used to extract information about the value or price of a product. The `description` attribute provides instructions to extract any sentences about the value or price and output them as a comma-separated Python list.

These `ResponseSchema` objects are useful for extracting specific types of information from text data, which can be used for further analysis or processing. By defining these objects, the code provides a clear and structured way to extract information from text data, which can help to improve the accuracy and efficiency of data processing tasks.

After you defined some properties with ResponseSchema you can create a definition for your json output.

In [63]:

answer_schema = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(answer_schema)
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


The code above defines an array of `ResponseSchema` objects named `answer_schema`, which are used to extract specific information from text data. The `answer_schema` array contains three `ResponseSchema` objects named `gift_schema`, `delivery_days_schema`, and `price_value_schema`.

The `output_parser` variable is then defined as a `StructuredOutputParser` object that is created from the `answer_schema` array. This object is used to parse the text data and extract the information specified in the `answer_schema` array.

The `get_format_instructions()` method is then called on the `output_parser` object to generate format instructions for the extracted information. These format instructions provide guidance on how to format the extracted information for further processing or analysis.

Finally, the generated format instructions are printed to the console using the `print()` function. This allows the user to see the format instructions and use them to further process or analyze the extracted information.

Overall, this code provides a structured way to extract specific information from text data and generate format instructions for that information. This can be useful for a variety of data processing and analysis tasks, and can help to improve the accuracy and efficiency of those tasks.

In [64]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [65]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "

In [67]:
response = chat(messages)
pprint(response.content)

('```json\n'
 '{\n'
 '\t"gift": true,\n'
 '\t"delivery_days": "2",\n'
 '\t"price_value": ["It\'s slightly more expensive than the other leaf blowers '
 'out there, but I think it\'s worth it for the extra features."]\n'
 '}\n'
 '```')


Output_parser can be convenient used to parse the output.

In [68]:
output_dict = output_parser.parse(response.content)
output_dict
type(output_dict)
pprint(output_dict)

{'delivery_days': '2',
 'gift': True,
 'price_value': ["It's slightly more expensive than the other leaf blowers out "
                 "there, but I think it's worth it for the extra features."]}


In [69]:
output_dict.get('delivery_days')

'2'